In [31]:
from distutils.log import error
from open_crypto import runner
from matplotlib import pyplot as plt
import sqlite3 
import pandas as pd
import datetime

#params

depths_ranges = [50,100,150,200]

#create database
depth_db = pd.DataFrame(columns=['exchange', 'mid price', 'spread', 'depth','depth units','depth units USD', 'time'])

In [33]:
#fetch data from exchanges
def check_dbstatus():
    if con.total_changes == 0:
        return(print("DB connection established"))
    else:
        return(print("failed to establish connection to database"))

#fetch data from exchanges
runner.run('amkt.yaml')
# Read sqlite query results into a pandas DataFrame

con = sqlite3.connect("amkt_data.db")

check_dbstatus()

orderbook = pd.read_sql_query('SELECT * FROM order_books_view;', con)
prices = pd.read_sql_query('SELECT exchange, MAX(price) AS price FROM trades_view GROUP BY exchange;', con)
# Verify that result of SQL query is stored in the dataframe

#get the time of the snapshot
cur_time = datetime.datetime.now() 

con.close()

def calculate_depth(database, exchange, range): #depths are calculated in BPS
    range = range / 100 #convert to kBPS (1000BPS = 1KBPS)
    #get midprice of the last range of trades
    price = prices.loc[prices['exchange'] == exchange, 'price'].item()

    #calculate depth boundaries
    depth_asks = price * (1 + range)
    depth_bids = price - (price * range)

    #get trades within the depth boundaries
    trades = database.loc[(database['bids_price'] > depth_bids) & (database['asks_price'] < depth_asks)]
    trades = trades.assign(**{'price': trades['bids_price'].fillna(trades['asks_price'])})
    

    #calculate depth
    depth = trades['price'].std()

    return depth

def calculate_spread(database, exchange):
    #get midprice of the last range of trades
    bidmaxprice = orderbook.loc[database['exchange'] == exchange, database['bids_price'].min()]
    askminprice = orderbook.loc[database['exchange'] == exchange, database['asks_price'].max()]

    #calculate spread
    spread = bidmaxprice - askminprice

    return spread

for exchange in orderbook['exchange'].unique():
    for depth_val in depths_ranges:
        df = pd.DataFrame()
        depth_units = calculate_depth(orderbook, exchange, depth_val)
        spread = calculate_spread(orderbook, exchange)
        depth_units_USD = depth_units * prices.loc[prices['exchange'] == exchange, 'price'].item()
        row = [exchange, prices.loc[prices['exchange'] == exchange, 'price'].item(), spread, depth_val, depth_units, depth_units_USD, cur_time] 
        depth_db.loc[len(depth_db)] = row

Requesting 2 exchange(s) for job: orderbook.                                    
Requesting 2 exchange(s) for job: tickers.
Requesting data... 50.00 % /                                                    Pair-ID 336 - Coinbase: 17222 tuple(s)
Pair-ID 661 - Ftx: 100 tuple(s)
Terminating.
Pair-ID 336 - Coinbase: 383 tuple(s)                                            
Pair-ID 661 - Ftx: 100 tuple(s)
Terminating.
DB connection established


NotImplementedError: iLocation based boolean indexing on an integer type is not available

In [ ]:
depth_db

,exchange,mid price,spread,depth,depth units,depth units USD,time
